# Data Preparation 

In [1]:
import helper
#import pandas as pd

2023-07-30 18:54:07.325250: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 18:54:08.349593: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# Openstax Biology
topic = 'openstax_biology'
docs = helper.openstax_to_doc(path='data/openstax_biology.csv')

In [6]:
len(docs)

205

In [7]:
doc_store = helper.add_to_docstore(docs, index=topic)
doc_store = helper.classify_docs(labels=['physics', 'chemistry', 'biology'],
                                doc_store=doc_store, index=topic)

/home/rjt/miniconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Classifying documents:  78%|███████▊  | 160/205 [00:16<00:04,  9.98it/s]/home/rjt/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Classifying documents: 100%|██████████| 205/205 [00:20<00:00,  9.80it/s]


In [32]:
'''
bio_docs = doc_store.get_all_documents(index='openstax_biology')
for _, doc in enumerate(bio_docs):
    if doc.meta['classification']['label'] != 'biology':
        print(_, doc.content)
'''

2 Proteins, carbohydrates, nucleic acids, and lipids are the four major classes of biological macromolecules—large molecules necessary for life that are built from smaller organic molecules. Macromolecules are comprised of single units scientists call monomers that are joined by covalent bonds to form larger polymers. The polymer is more than the sum of its parts: it acquires new characteristics, and leads to an osmotic pressure that is much lower than that formed by its ingredients. This is an important advantage in maintaining cellular osmotic conditions. A monomer joins with another monomer with water molecule release, leading to a covalent bond forming. Scientists call these dehydration or condensation reactions. When polymers break down into smaller units (monomers), they use a water molecule for each bond broken by these reactions. Such reactions are hydrolysis reactions. Dehydration and hydrolysis reactions are similar for all macromolecules, but each monomer and polymer reactio

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(index=topic)


In [ ]:
docudocument_store.get_document_count(index=topic)

In [ ]:
from haystack.nodes import TransformersDocumentClassifier
classifier = TransformersDocumentClassifier(task='zero-shot-classification',
                                            model_name_or_path='MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli',
                                            labels=['physics', 'chemistry', 'biology'], use_gpu=True)

In [ ]:
docs = document_store.get_all_documents(index=topic)

In [ ]:
x = classifier.predict(docs)

In [ ]:
document_store.write_documents(x, index=topic)

In [ ]:
y = document_store.get_all_documents(index=topic)

In [ ]:
y[0]

In [ ]:
#document_store.get_all_documents(index=topic)

In [ ]:
import pandas as pd
df = pd.read_csv('data/openstax_biology.csv')
df['summary_heading'] = df['summary_heading'].replace('\d+\.\d+ ', '', regex=True)
df.columns = ['topic', 'content', 'subject']
df['meta'] = df.apply(lambda x: {'topic': x['topic'], 'subject': x['subject']}, axis=1).to_list()
#dict_df = {'content': df['content'], 'meta':df['metadata']}
df = df[['content', 'meta']]
x = df.to_dict('records')

In [ ]:
x[:2]

In [ ]:
dict_df = df.to_dict(orient='records')

In [ ]:
type(dict_df)